# __New Tourist Offices in Rome__

The city of Rome is planning to open five new tourist offices in Rome, which specialize in *long* guided walking tours of historic sites. The city is interested in the optimal placement of these five tourist offices.

Ample funding is available, making differences in rent and housing prices insignificant for our purposes. Since these tours are *walking* tours, availability (and quality) of public transport connecting different sights and tourist offices is insignificant for our purposes as well.

The problem then becomes to find five locations, so that each historic site in Rome has low (walking) distance to at least one of the locations.

## Data Collection, Visualization and Preparation

First we import all the necessary libraries:

In [1]:
!conda install -c conda-forge folium=0.11.0 --yes

import json
import math
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

Solving environment: done

# All requested packages already installed.



We will use the Foursquare API to obtain the necessary location data. We need credentials in order to do that:

In [32]:
CLIENT_ID = '' # removed for privacy reasons
CLIENT_SECRET = '' # removed for privacy reasons
VERSION = '20200808'

Rome is subdivided into 15 so called "municipi". First we will look for the 50 most popular historic sites in all of Rome (we search with a large radius). Afterwards we will look for the 50 most popular historic sites in each of the municipi (we search with a smaller radius). This is necessary because the Foursquare API never returns more than 50 venues per call.

We start with the 50 most popular historic sites in all of Rome:

In [3]:
# url for the GET request

url = 'https://api.foursquare.com/v2/venues/explore?&clie\
nt_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        'Rome, Italy', 
        25000, # searching in radius of 25km
        50, # the 50 most popular historic sites
        '4deefb944765f83613cdba6e') # category ID for "Historic Site"

In [4]:
# GET request

results = requests.get(url).json()["response"]['groups'][0]['items']

In [5]:
# for each result, we store the name of the venue and its geographical coordinates in a list (of tuples)

historic_sites_list = []

for res in results:
    historic_sites_list = historic_sites_list + [(res['venue']['name'],
                                                  res['venue']['location']['lat'],
                                                  res['venue']['location']['lng'])]

Now we look for the 50 most popular historic sites in each of the municipi. We start by collecting the coordinates of the municipi:

In [6]:
# coordinates of the municipi, scraped from wikipedia

municipi_coord  = pd.DataFrame([('I', 41.893056, 12.482778),
                                ('II', 41.929958, 12.518931),
                                ('III', 41.936081, 12.535117),
                                ('IV', 41.933492, 12.598747),
                                ('V', 41.890664, 12.548489),
                                ('VI', 41.869658, 12.632731),
                                ('VII', 41.8817, 12.5228),
                                ('VIII', 41.841228, 12.484289),
                                ('IX', 41.814878, 12.479981),
                                ('X', 41.847328, 12.280531),
                                ('XI', 41.855283, 12.494761),
                                ('XII', 41.8761, 12.4501),
                                ('XIII', 41.899142, 12.424158),
                                ('XIV', 41.940964, 12.418628),
                                ('XV', 41.955436, 12.48485)],
                               columns = ['Municipio', 'Latitude', 'Longitude'])

We display the municipi on a map:

In [7]:
# coordinates of Rome scraped from wikipedia

map_Rome = folium.Map(location=[41.883333, 12.5], zoom_start=11)

for lat, lng, mun in zip(municipi_coord['Latitude'], municipi_coord['Longitude'],municipi_coord['Municipio']):
    label = folium.Popup('Municipio {}; coordinates: {}, {}'.format(mun, lat, lng), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Rome)  
    
map_Rome

### <font color=red>__Folium Maps tend to not show up on GitHub. Look at the image sub1.png in the GitHub folder.__</font>

Now we are ready to obtain the 50 most popular historic sites for each of the municipi:

In [8]:
latitudes = municipi_coord['Latitude']
longitudes = municipi_coord['Longitude']

venues_list=[]
for lat, lng in zip(latitudes, longitudes):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&clie\
nt_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat,
        lng,
        2000, # searching in 2km radius
        50, # default limit of Foursquare API, cannot be exceeded
        '4deefb944765f83613cdba6e') # category ID for "Historic Site"
            
    # make the GET request
    try:
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # we add the relevant information of all the values to our list
        for res in results:
            historic_sites_list = historic_sites_list + [(res['venue']['name'],
                                                          res['venue']['location']['lat'],
                                                          res['venue']['location']['lng'])]
    except:
        pass

We now create a dataframe from our list of historic sites:

In [9]:
historic_sites = pd.DataFrame(historic_sites_list, columns = ['Historic Site', 'Latitude', 'Longitude'])

We briefly inspect our dataframe, to see whether the results are as expected (one of course expects the Colosseum and the Forum Romanum to be found at the top of this dataframe):

In [10]:
historic_sites.head(10)

,Historic Site,Latitude,Longitude
0,Colosseo,41.890633,12.492378
1,Foro Romano,41.892030,12.487037
2,Foro di Cesare,41.894128,12.485232
3,Palatino,41.888234,12.487209
4,Stadio palatino (Stadio palatino | Stadio di D...,41.887582,12.487496
5,Foro di Traiano,41.894729,12.484871
6,Terme di Caracalla,41.878990,12.492443
7,Portico d'Ottavia,41.892382,12.478500
8,Scalinata di Trinità dei Monti,41.905974,12.482647
9,Foro di Augusto,41.894070,12.486483


Since we carried out several GET requests, it is possible that several historic sites are listed more than once. We remove all duplicates:

In [11]:
print(historic_sites.shape)
historic_sites.drop_duplicates(subset = 'Historic Site', keep = 'first', inplace = True)
print(historic_sites.shape)

(119, 3)
(83, 3)


We display all the historic sites in our dataframe on a map:

In [12]:
# coordinates of Rome scraped from wikipedia

map_historic_sites = folium.Map(location=[41.883333, 12.5], zoom_start=12)

for lat, lng, site in zip(historic_sites['Latitude'], historic_sites['Longitude'], historic_sites['Historic Site']):
    label = folium.Popup('{}'.format(site), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_historic_sites)  
    
map_historic_sites

### <font color=red>__Folium Maps tend to not show up on GitHub. Look at the image sub2.png in the GitHub folder.__</font>

After manually inspecting the outliers, we remove the insignificant ones:

In [13]:
print(historic_sites.shape)
historic_sites = historic_sites[(historic_sites['Historic Site'] != 'Barbiellini Amidei') & \
                                (historic_sites['Historic Site'] != 'Palazzo degli Uffici di EUR Spa')]
print(historic_sites.shape)

(83, 3)
(81, 3)


We reset the index and have a look at the dataframe:

In [14]:
historic_sites.reset_index(inplace = True, drop = True)
historic_sites.head(10)

,Historic Site,Latitude,Longitude
0,Colosseo,41.890633,12.492378
1,Foro Romano,41.892030,12.487037
2,Foro di Cesare,41.894128,12.485232
3,Palatino,41.888234,12.487209
4,Stadio palatino (Stadio palatino | Stadio di D...,41.887582,12.487496
5,Foro di Traiano,41.894729,12.484871
6,Terme di Caracalla,41.878990,12.492443
7,Portico d'Ottavia,41.892382,12.478500
8,Scalinata di Trinità dei Monti,41.905974,12.482647
9,Foro di Augusto,41.894070,12.486483


We display the historic sites we will work with on a map:

In [15]:
# coordinates of Rome scraped from wikipedia

map_historic_sites = folium.Map(location=[41.883333, 12.5], zoom_start=12)

for lat, lng, site in zip(historic_sites['Latitude'], historic_sites['Longitude'], historic_sites['Historic Site']):
    label = folium.Popup('{}'.format(site), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_historic_sites)  
    
map_historic_sites

### <font color=red>__Folium Maps tend to not show up on GitHub. Look at the image sub3.png in the GitHub folder.__</font>

## Methods and Analysis

In order to compute distances, we first rewrite the coordinates in __radians__:

In [16]:
historic_sites['Latitude'] *= math.pi/180
historic_sites['Longitude'] *= math.pi/180

In order to compute distances between locations, we need to obtain a formula in terms of the latitude and longitude. Using (a slight variation of) spherical coordinates, one easily arrives at the haversine formula: assuming that (latitude, longitude) of two points are $(\phi_1 , \lambda_1 )$ and $(\phi_2 , \lambda_2 )$, the distance of these two points ("along the surface of the earth", i.e., not the distance in 3-dimensional space, but rather the distance when moving along the surface) computes to:
\begin{align*}
2R\operatorname{arcsin}\left({\sqrt{\sin^2\left(\frac{\phi_2 -\phi_1}{2}\right)+\cos (\phi_1 )\cos (\phi_2 )\sin^2\left(\frac{\lambda_2 -\lambda_1}{2}\right)}}\right)\text{,}
\end{align*}
where $R$ is the radius of the earth.

Noting, however, that we can restrict attention to points close to Rome (coordinates: $(\phi_0 ,\lambda_0 )=(41.883333\cdot \pi /180, 12.5\cdot \pi /180)$, in radians), the quantities $|\phi_2 -\phi_1 |$, $|\phi_2 -\phi_0 |$, $|\phi_1 -\phi_0 |$ and $|\lambda_2 -\lambda_1 |$ will always be very small, whenever $(\phi_1 , \lambda_1 )$ and $(\phi_2 , \lambda_2 )$ are the coordinates (in radians) of two points close to Rome. That implies that the following is a good approximation for the distances between points close to Rome (use Taylor series to see this):

\begin{align*}
& 2R\sqrt{\left(\frac{\phi_2 -\phi_1}{2}\right)^2+\cos^2 (\phi_0 )\cdot\left(\frac{\lambda_2 -\lambda_1}{2}\right)^2}\\
= &R\cdot\Vert{(\phi_1 ,\lambda_1\cos (\phi_0 ) )- (\phi_2 ,\lambda_2\cos (\phi_0 ) )}\Vert \text{,}
\end{align*}
where $\Vert\cdot\Vert$ denotes the Euclidean norm ($2$-norm) on $\mathbb{R}^2$.

For every longitude value $\lambda$ (in radians), we define the quantity ${\lambda}'=\lambda\cdot\cos (\phi_0 )$. With this notation, the upper mentioned approximation for the distances becomes:
\begin{align*}
R\cdot\Vert{(\phi_1 ,{\lambda}'_1)- (\phi_2 ,{\lambda}'_2)}\Vert \text{.}
\end{align*}
Since $R$ is just a constant (positive) factor, and we are only interested in __comparing__ distances, we will drop that factor and consider the following expression instead of the distance:
\begin{align*}
\Vert{(\phi_1 ,{\lambda}'_1)- (\phi_2 ,{\lambda}'_2)}\Vert \text{.}
\end{align*}

But that is just the __Euclidean distance__ (with respect to the modified coordinates $(\phi ,{\lambda}' )$).

What we have gained from this is that we can now use the K-Means algorithm, which uses Euclidean distance. Currently, in our dataframe, the (latitude, longitude) pairs are of the form $(\phi ,\lambda )$, so we have to transform them to the modified coordinates $(\phi ,{\lambda}' )$:

In [17]:
historic_sites['Longitude'] *= math.cos(41.883333*(math.pi/180))
historic_sites.columns = ['Historic Site', 'Latitude (rad)', 'modified Longitude']

In [18]:
historic_sites.head()

,Historic Site,Latitude (rad),modified Longitude
0,Colosseo,0.731129,0.162327
1,Foro Romano,0.731154,0.162258
2,Foro di Cesare,0.731190,0.162234
3,Palatino,0.731088,0.162260
4,Stadio palatino (Stadio palatino | Stadio di D...,0.731076,0.162263


Now we can use the K-Means algorithm, where the number of clusters coincides with the number of planned new tourist offices:

In [19]:
df_clustering = historic_sites.drop('Historic Site', axis = 1) # we remove the column containing strings

kmeans = KMeans(n_clusters = 5, # the number of tourist centers we want to build
                random_state=0).fit(df_clustering) 

We are interested in the centers of the five clusters, since those will be the locations for the new tourist offices. Noting that the centers were computed using the modified coordinates, we transform them back to the usual Latitude and Longitude values.

In [20]:
centers = pd.DataFrame(kmeans.cluster_centers_, columns = ['Latitude', 'Longitude'])

# transforming back to usual latitude and longitude values
centers['Latitude'] *= 180/math.pi
centers['Longitude'] *= 180/( math.pi * math.cos(41.883333*(math.pi/180)) )

In [21]:
centers

,Latitude,Longitude
0,41.865195,12.507724
1,41.891783,12.487530
2,41.930894,12.521352
3,41.900684,12.472790
4,41.878225,12.548237


Now, of course, we want to visualize how the various historic sites are located in relation to the closest new tourist office. 

In order to do that, we create a dataframe that contains all the historic sites together with their respective cluster labels. We also have to transform the Latitude (in radians) and the "modified Longitude" back to their original values:

In [22]:
historic_sites_clustered = historic_sites.copy(deep = True)
historic_sites_clustered.columns = ['Historic Site', 'Latitude', 'Longitude']

# adding a column with the cluster labels
historic_sites_clustered['Cluster'] = kmeans.labels_

# transforming the coordinates back to the old values
historic_sites_clustered['Latitude'] *= 180/math.pi
historic_sites_clustered['Longitude'] *= 180/( math.pi * math.cos(41.883333*(math.pi/180)) )

In [23]:
historic_sites_clustered.head(3)

,Historic Site,Latitude,Longitude,Cluster
0,Colosseo,41.890633,12.492378,1
1,Foro Romano,41.892030,12.487037,1
2,Foro di Cesare,41.894128,12.485232,1


We now display the centers and the historic sites on a map together. For visualization purposes we choose different colors for different clusters, and furthermore choose a more subtle Folium tile style.

In [24]:
map_complete = folium.Map(location=[41.89, 12.5], zoom_start=13, tiles = 'CartoDB positron')

colors = ['#8000ff', '#00b5eb', 'brown', '#ffb360', 'green']

for lat, lng, site, cluster in zip(historic_sites_clustered['Latitude'],
                                   historic_sites_clustered['Longitude'],
                                   historic_sites_clustered['Historic Site'],
                                   historic_sites_clustered['Cluster']):
    label = folium.Popup(str(site), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color=colors[int(cluster)],
        fill=True,
        fill_color=colors[int(cluster)],
        fill_opacity=0.7).add_to(map_complete)   
    
for lat, lng in zip(centers['Latitude'], centers['Longitude']):
    label = folium.Popup('Tourist Office', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5.5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_complete)

map_complete

### <font color=red>__Folium Maps tend to not show up on GitHub. Look at the image sub4.png in the GitHub folder.__</font>

Looking at the above map, it seems like we were successful. However, we still need to verify that each historic site is within walking distance of the closest tourist office.

We first define a function that calculates the approximate distance between two points (close to Rome), where the points are given by their respective latitude and longitude values:

In [25]:
R = 6371000 # radius of earth, in meters

def dist_points(lat_1, lng_1, lat_2, lng_2):
    
    # we transform to modified coordinates
    lat_1 *= math.pi / 180
    lat_2 *= math.pi / 180
    lng_1 *= ( math.pi * math.cos(41.883333*(math.pi/180)) ) / 180
    lng_2 *= ( math.pi * math.cos(41.883333*(math.pi/180)) ) / 180
    
    # recall our approximation formula for distances between points close to Rome (in terms of the modified coordinates)
    distance = R * math.sqrt( (lat_2 - lat_1) ** 2 + (lng_2 - lng_1) ** 2 )
    
    return distance

Since, according to the scikit-learn documentation, cluster_centers_ and labels_ are not necessarily consistent, we will calculate the distance to all cluster centers separately and take the minimum.

We will define a function that does just that; for points close to Rome (once again given in terms of latitude and longitude):

In [26]:
def min_cluster_dist(lat, lng):
    distances = []
    
    # calculate the distance of the point to each of the centers and add the result to the "distances" list
    for i in range(5):
        distances = distances + [dist_points(lat,
                                             lng,
                                             centers['Latitude'][i],
                                             centers['Longitude'][i])]
    
    return min(distances) # distance to the closest center(s)

We will do that for all our historic sites and store the result in our dataframe:

In [27]:
historic_sites_clustered['Distance to nearest Tourist Office (in meters)'] = \
historic_sites_clustered.apply(lambda row : min_cluster_dist(row['Latitude'],
                                                             row['Longitude']), axis=1)

In [28]:
historic_sites_clustered.head(10)

,Historic Site,Latitude,Longitude,Cluster,Distance to nearest Tourist Office (in meters)
0,Colosseo,41.890633,12.492378,1,421.221871
1,Foro Romano,41.892030,12.487037,1,49.141233
2,Foro di Cesare,41.894128,12.485232,1,322.711627
3,Palatino,41.888234,12.487209,1,395.603485
4,Stadio palatino (Stadio palatino | Stadio di D...,41.887582,12.487496,1,467.144174
5,Foro di Traiano,41.894729,12.484871,1,394.599467
6,Terme di Caracalla,41.878990,12.492443,1,1479.559773
7,Portico d'Ottavia,41.892382,12.478500,1,750.527376
8,Scalinata di Trinità dei Monti,41.905974,12.482647,3,1005.930716
9,Foro di Augusto,41.894070,12.486483,1,268.664037


We have computed the distance to the nearest tourist office(s) for each historic site in our dataframe. We now determine the largest such distance:

In [29]:
historic_sites_clustered['Distance to nearest Tourist Office (in meters)'].max()

2068.9441082205894

So each of the historic sites has distance less than 2.1 km to at least one of the tourist offices. While, of course, the path one has to take is usually longer than the geometric distance (due to the layout/terrain of the city), the corresponding factor rarely exceeds 1.5, giving us an estimated longest path of roughly 3 km.

We infer that each historic site is within walking distance of at least one of the tourist offices, as desired.

## Discussion and Conclusion

Before we make our final recommendation, we display the potential new tourist offices on a map:

In [30]:
# coordinates of Rome scraped from wikipedia

map_centers = folium.Map(location=[41.883333, 12.5], zoom_start=12)

for lat, lng in zip(centers['Latitude'], centers['Longitude']):
    label = folium.Popup('Tourist Office', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_centers)  
    
map_centers

### <font color=red>__Folium Maps tend to not show up on GitHub. Look at the image sub5.png in the GitHub folder.__</font>

One readily verifies that all five potential new offices are conveniently located by manually checking public transport schedules. Furthermore, the fact that two of the new offices are quite central (and hence close to each other) is advantageous, since the density of historic sites and tourists is higher in the centrum. As established in the previous section, none of the historic sites is longer than a 3 km walk away from the closest tourist office(s).

In conclusion, we recommend that the city of Rome establishes the five new tourist offices in the following locations: 

In [31]:
centers

,Latitude,Longitude
0,41.865195,12.507724
1,41.891783,12.487530
2,41.930894,12.521352
3,41.900684,12.472790
4,41.878225,12.548237
